In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("Environment variables loaded successfully!")
print(f"JINA_API_KEY loaded: {'✅' if os.getenv('JINA_API_KEY') else '❌'}")
print(f"OPENROUTER_API_KEY loaded: {'✅' if os.getenv('OPENROUTER_API_KEY') else '❌'}")

In [ ]:

# Contoh Pencarian dengan FAISS dan Embedding Jina AI




import faiss
import numpy as np
import json
import requests

# Load FAISS index
faiss_index = faiss.read_index("faiss_index")

# (Opsional) Load metadata jika ingin tahu mapping ke dokumen/chunk
with open("faiss_metadata.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)

# Fungsi untuk mendapatkan embedding dari Jina AI
def get_jina_embedding(text, api_key, model="jina-embeddings-v3"):
    url = "https://api.jina.ai/v1/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": model,
        "task": "text-matching",
        "late_chunking": False,
        "truncate": False,
        "input": [text]
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json().get("data", [])
        if result and "embedding" in result[0]:
            return result[0]["embedding"]
    print("Jina API error:", response.status_code, response.text)
    return None

# Contoh query
query = "bagaimana cara membuat KTP ?"

# Dapatkan embedding query dari Jina AI
api_key = "jina_028af8dd31a34f0381944f463098db25cG6BK9grb5PbjmtxfMRHwjsB4sRu"
query_emb = get_jina_embedding(query, api_key)
if query_emb is None:
    raise Exception("maaf, saya tidak mengetahuinya")

# Ubah ke numpy array dan reshape
query_emb_np = np.array(query_emb, dtype="float32").reshape(1, -1)

# Cari top-5 dokumen terdekat di FAISS
D, I = faiss_index.search(query_emb_np, k=10)
print("Top 5 index:", I)
print("Top 5 distance:", D)

# Ambil metadata dan tampilkan hasil
for idx in I[0]:
    print(metadata[idx])

# (Opsional) Menyusun konteks untuk LLM dari hasil FAISS
context = "\n\n".join([str(metadata[idx]) for idx in I[0]])

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

llm = ChatOpenAI(
    openai_api_key="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",
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="deepseek/deepseek-r1-0528:free"
)

messages = [
    SystemMessage(content="Jawablah hanya berdasarkan informasi berikut. Jika tidak ada info relevan, jawab: 'Maaf, saya tidak tahu.'"),
    SystemMessage(content=f"Konteks dokumen:\n{context}"),
    HumanMessage(content=query)
]

response = llm(messages)
print("Jawaban LLM:", response.content)


Top 5 index: [[ 93  95  92  94  91  90 301  89  81  88]]
Top 5 distance: [[0.9816766 1.0036747 1.0038304 1.0537189 1.0825939 1.0993018 1.1001995
  1.1132741 1.1198411 1.1229044]]
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 93}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 95}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 92}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 94}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 91}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 90}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 1, 'chunk_index': 145}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 89}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 81}
{'filename': 'Perda

In [1]:

# Contoh Pencarian dengan FAISS dan Embedding Jina AI (Batch Embedding)

import faiss
import numpy as np
import json
import requests

# Load FAISS index
faiss_index = faiss.read_index("faiss_index")

# (Opsional) Load metadata jika ingin tahu mapping ke dokumen/chunk
with open("faiss_metadata.json", "r", encoding="utf-8") as f:
    metadata = json.load(f)

# Fungsi untuk mendapatkan batch embedding dari Jina AI
def get_jina_batch_embedding(texts, api_key, model="jina-embeddings-v3"):
    url = "https://api.jina.ai/v1/embeddings"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": model,
        "task": "retrieval.passage",
        "late_chunking": False,
        "truncate": False,
        "input": texts
    }
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        result = response.json().get("data", [])
        return [item["embedding"] for item in result if "embedding" in item]
    print("Jina API error:", response.status_code, response.text)
    return None

# Contoh batch query
queries = [
    "bagaimana cara membuat KTP ?",
    "prosedur pembayaran pajak kendaraan",
    "syarat mengurus akta kelahiran"
]

# Dapatkan batch embedding query dari Jina AI
api_key = "jina_028af8dd31a34f0381944f463098db25cG6BK9grb5PbjmtxfMRHwjsB4sRu"
query_embs = get_jina_batch_embedding(queries, api_key)
if not query_embs:
    raise Exception("maaf, saya tidak mengetahuinya")

# Ubah ke numpy array dan reshape
query_embs_np = np.array(query_embs, dtype="float32")

# Cari top-10 dokumen terdekat di FAISS untuk setiap query
for i, query_emb_np in enumerate(query_embs_np):
    D, I = faiss_index.search(query_emb_np.reshape(1, -1), k=10)
    print(f"\nQuery: {queries[i]}")
    print("Top 10 index:", I)
    print("Top 10 distance:", D)
    # Ambil metadata dan tampilkan hasil
    for idx in I[0]:
        print(metadata[idx])
    # (Opsional) Menyusun konteks untuk LLM dari hasil FAISS
    context = "\n\n".join([str(metadata[idx]) for idx in I[0]])

    from langchain_openai import ChatOpenAI
    from langchain.schema import SystemMessage, HumanMessage

    llm = ChatOpenAI(
        openai_api_key="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",
        openai_api_base="https://openrouter.ai/api/v1",
        model_name="deepseek/deepseek-r1-0528:free"
    )

    messages = [
        SystemMessage(content="Jawablah hanya berdasarkan informasi berikut. Jika tidak ada info relevan, jawab: 'Maaf, saya tidak tahu.'"),
        SystemMessage(content=f"Konteks dokumen:\n{context}"),
        HumanMessage(content=queries[i])
    ]

    response = llm(messages)
    print("Jawaban LLM:", response.content)



Query: bagaimana cara membuat KTP ?
Top 10 index: [[ 95  93  92  94  91  90  88  89 106 301]]
Top 10 distance: [[1.3126535 1.3311918 1.3343139 1.3721619 1.3989358 1.3989913 1.4018297
  1.4195609 1.4255455 1.4320018]]
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 95}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 93}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 92}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 94}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 91}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 90}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 88}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 'chunk_index': 89}
{'filename': 'Perda Kota Cimahi No. 8 Tahun 2014.pdf', 'doc_part': 0, 

C:\Users\Willy Hanafi\AppData\Local\Temp\ipykernel_8484\2805385529.py:79: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


Jawaban LLM: Berdasarkan Peraturan Daerah Kota Cimahi No. 8 Tahun 2014:  

1. **Dasar Hukum**:  
   Penduduk wajib memiliki Kartu Tanda Penduduk (KTP) sebagai bukti identitas yang dikeluarkan setelah pencatatan biodata dan penerbitan Nomor Induk Kependudukan (NIK) (Pasal 15).  

2. **Proses Pencatatan Biodata**:  
   Pencatatan biodata penduduk untuk memperoleh NIK dilakukan pada kondisi berikut (Pasal 15 Ayat 1):  
   - Saat kelahiran.  
   - Saat melaporkan peristiwa kependudukan (seperti pindah domisili atau perubahan status).  
   - **Saat berusia 17 tahun atau sudah pernah menikah** (kondisi khusus penerbitan KTP).  
   - Saat ada perubahan nama atau biodata.  

3. **Kewajiban Pelaporan**:  
   Penduduk wajib melaporkan peristiwa kependudukan (termasuk pembuatan KTP) kepada Instansi Pelaksana (Dinas Dukcapil) dengan memenuhi persyaratan yang ditetapkan (Pasal 12 Ayat 1 dan 2).  

4. **Persyaratan**:  
   Meskipun tidak dirinci secara spesifik dalam dokumen yang tersedia, persyarat